# METHODOLOGY - Spatial join of climate station data to CMA boundary data

This notebook is a walkthrough on how I connected each climate station to a particular CMA or CA. For this analysis, we use Pandas and Geopandas.

In [28]:
import pandas as pd
import geopandas

Next, we import the 2021 boundary files from Statistics Canada, and convert them to EPSG:4326 (lat/lon).

In [29]:
boundariesUrl = "https://www12.statcan.gc.ca/census-recensement/2021/geo/sip-pis/boundary-limites/files-fichiers/lcma000b21a_e.zip"

boundaries = geopandas.read_file(boundariesUrl).to_crs("EPSG:4326")

boundaries.head(3)

,CMAUID,CMAPUID,DGUID,DGUIDP,CMANAME,CMATYPE,LANDAREA,PRUID,geometry
0,001,10001,2021S0503001,None,St. John's,B,931.5637,10,"MULTIPOLYGON (((-52.97228 47.51126, -52.97243 ..."
1,010,10010,2021S0504010,None,Grand Falls-Windsor,D,54.8429,10,"POLYGON ((-55.61289 48.93946, -55.61332 48.939..."
2,011,10011,2021S0504011,None,Gander,D,2412.6700,10,"POLYGON ((-54.28994 48.96605, -54.28319 48.962..."


Now, we import a list of climate stations from Environment and Climate Change Canada. It can be found in [this Google Drive](https://drive.google.com/drive/folders/1WJCDEU34c60IfOnG4rv5EPZ4IhhW9vZH).

In [30]:
stations = pd.read_csv("../data/raw_stations.csv", skiprows=3)

stations.head(3)

,Name,Province,Climate ID,Station ID,WMO ID,TC ID,Latitude (Decimal Degrees),Longitude (Decimal Degrees),Latitude,Longitude,Elevation (m),First Year,Last Year,HLY First Year,HLY Last Year,DLY First Year,DLY Last Year,MLY First Year,MLY Last Year
0,ACTIVE PASS,BRITISH COLUMBIA,1010066,14,NaN,NaN,48.87,-123.28,485200000.0,-1.231700e+09,4.0,1984,1996,NaN,NaN,1984.0,1996.0,1984.0,1996.0
1,ALBERT HEAD,BRITISH COLUMBIA,1010235,15,NaN,NaN,48.40,-123.48,482400000.0,-1.232900e+09,17.0,1971,1995,NaN,NaN,1971.0,1995.0,1971.0,1995.0
2,BAMBERTON OCEAN CEMENT,BRITISH COLUMBIA,1010595,16,NaN,NaN,48.58,-123.52,483500000.0,-1.233100e+09,85.3,1961,1980,NaN,NaN,1961.0,1980.0,1961.0,1980.0


Then, we convert our Latitude/Longitude columsn to a geometry column.

In [31]:
stations["geometry"] = geopandas.points_from_xy(stations["Longitude (Decimal Degrees)"], stations["Latitude (Decimal Degrees)"])

Then convert the pandas dataframe to a geodataframe, and set the CRS to EPSG:4326.

In [32]:
stations = geopandas.GeoDataFrame(stations).set_crs("EPSG:4326")

stations.head(3)

,Name,Province,Climate ID,Station ID,WMO ID,TC ID,Latitude (Decimal Degrees),Longitude (Decimal Degrees),Latitude,Longitude,Elevation (m),First Year,Last Year,HLY First Year,HLY Last Year,DLY First Year,DLY Last Year,MLY First Year,MLY Last Year,geometry
0,ACTIVE PASS,BRITISH COLUMBIA,1010066,14,NaN,NaN,48.87,-123.28,485200000.0,-1.231700e+09,4.0,1984,1996,NaN,NaN,1984.0,1996.0,1984.0,1996.0,POINT (-123.28000 48.87000)
1,ALBERT HEAD,BRITISH COLUMBIA,1010235,15,NaN,NaN,48.40,-123.48,482400000.0,-1.232900e+09,17.0,1971,1995,NaN,NaN,1971.0,1995.0,1971.0,1995.0,POINT (-123.48000 48.40000)
2,BAMBERTON OCEAN CEMENT,BRITISH COLUMBIA,1010595,16,NaN,NaN,48.58,-123.52,483500000.0,-1.233100e+09,85.3,1961,1980,NaN,NaN,1961.0,1980.0,1961.0,1980.0,POINT (-123.52000 48.58000)


Now, we use a spatial join with the predicate "within". This will add columns for each point based on which CMA or CA boundary they fall within.

In [33]:
joined = stations.sjoin(boundaries, how="left", predicate='within')

joined.head(3)

,Name,Province,Climate ID,Station ID,WMO ID,TC ID,Latitude (Decimal Degrees),Longitude (Decimal Degrees),Latitude,Longitude,...,geometry,index_right,CMAUID,CMAPUID,DGUID,DGUIDP,CMANAME,CMATYPE,LANDAREA,PRUID
0,ACTIVE PASS,BRITISH COLUMBIA,1010066,14,NaN,NaN,48.87,-123.28,485200000.0,-1.231700e+09,...,POINT (-123.28000 48.87000),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ALBERT HEAD,BRITISH COLUMBIA,1010235,15,NaN,NaN,48.40,-123.48,482400000.0,-1.232900e+09,...,POINT (-123.48000 48.40000),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BAMBERTON OCEAN CEMENT,BRITISH COLUMBIA,1010595,16,NaN,NaN,48.58,-123.52,483500000.0,-1.233100e+09,...,POINT (-123.52000 48.58000),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now we specify which columns we want to keep, and toss any values that didn't fall within a city boundary.

In [34]:
columnsToKeep = ["Station ID", "Climate ID", "Name", "Latitude", "Longitude", "First Year", "Last Year", "CMAUID", "CMANAME", "PRUID"]

joined = joined.dropna(subset="index_right").loc[:, columnsToKeep].set_index("Climate ID")

joined.head(3)

,Station ID,Name,Latitude,Longitude,First Year,Last Year,CMAUID,CMANAME,PRUID
Climate ID,,,,,,,,,
1010774,18,BEAVER LAKE,483000000.0,-1.232100e+09,1894,1952,935,Victoria,59
1010960,20,BRENTWOOD BAY 2,483600000.0,-1.232800e+09,1987,1997,935,Victoria,59
1010961,21,BRENTWOOD CLARKE ROAD,483400000.0,-1.232700e+09,1972,1980,935,Victoria,59


We're also going to take the time to replace our PRUIDs with province abbreviations, so we only have to do this once.

In [35]:
prov_replace = {
    10: "NL",
    11: "PEI",
    12: "NS",
    13: "NB",
    24: "QC",
    35: "ON",
    46: "MB",
    47: "SK",
    48: "AB",
    59: "BC",
    60: "YK",
    61: "NWT",
    62: "NU",
}
        
joined["PRUID"] = joined["PRUID"].astype(int).replace(prov_replace)

Now we export for use in our app.

In [36]:
joined.to_csv("../data/stations_cma.csv")